# JQ Queries

The AIO Data Processor pipelines use JQ queries to transform the data. This notebook provides a brief introduction to JQ queries and how to use them. JQ queries are written in a simple and expressive language. They are easy to read and write. JQ queries can be used to extract data from JSON objects, filter data, and transform data.

You can use this notebook to learn how to write JQ queries and test them on sample data before using them in the AIO Data Processor pipelines. The notebook provides examples of JQ queries and explains how they work. You can use the examples as a starting point for writing your own JQ queries. For more information about JQ queries, see the [Azure IoT Operations JQ Expressions](./JQ_EXPRESSIONS.ipynb) notebook.

## Machine Status Calculation

In [1]:
export json='{
        "timestamp": "2023-12-18T07:08:51.9706158Z",
        "messageType": "ua-deltaframe",
        "payload": {
            "ns=2;s=AreaName.MachineName.Status.Idle": {
                "SourceTimestamp": "2023-12-18T07:08:51.5575029Z",
                "Value": true
            }
        },
        "dataSetWriterName": "SomeName",
        "sequenceNumber": 9001
}'

echo $json | jq

{
  "timestamp": "2023-12-18T07:08:51.9706158Z",
  "messageType": "ua-deltaframe",
  "payload": {
    "ns=2;s=AreaName.MachineName.Status.Idle": {
      "SourceTimestamp": "2023-12-18T07:08:51.5575029Z",
      "Value": true
    }
  },
  "dataSetWriterName": "SomeName",
  "sequenceNumber": 9001
}
  "timestamp": "2023-12-18T07:08:51.9706158Z",
  "messageType": "ua-deltaframe",
  "payload": {
    "ns=2;s=AreaName.MachineName.Status.Idle": {
      "SourceTimestamp": "2023-12-18T07:08:51.5575029Z",
      "Value": true
    }
  },
  "dataSetWriterName": "SomeName",
  "sequenceNumber": 9001
}


If you want to access the value of a specific field in a JSON object, you can use a JQ query to extract the value. For example, the following JQ query extracts the value of the `Value` field from the `$json` object:

In [2]:
echo $json | jq '.payload."ns=2;s=AreaName.MachineName.Status.Idle".Value'

true


The next cell takes the above JSON and extracts the value of the `payload` field and uses `to_entries` to convert the object into an array of key-value pairs.

In [3]:
export output=$(echo $json | jq '(.payload 
        | to_entries)')

echo $output | jq

[
  {
    "key": "ns=2;s=AreaName.MachineName.Status.Idle",
    "value": {
      "SourceTimestamp": "2023-12-18T07:08:51.5575029Z",
      "Value": true
    }
  }
]
  {
    "key": "ns=2;s=AreaName.MachineName.Status.Idle",
    "value": {
      "SourceTimestamp": "2023-12-18T07:08:51.5575029Z",
      "Value": true
    }
  }
]


Now that we have the key-value pairs, we can use the `map` function to transform the array. The below jq script does the following: 

1. `.payload`: This selects the `payload` field of the input JSON.

1. `to_entries`: This converts the JSON object into an array of key-value pairs.

1. `map()`: This function iterates over each key-value pair in the array.

1. `if .key | test("Status") then`: This checks if the key contains the string "Status".

1. `{"key": "NodeId", "value": .key}`: If the key contains "Status", it creates a new key-value pair where the key is "NodeId" and the value is the original key.

1. `{"key": (.key | split(".")[-1]), "value": .value.Value}`: If the key does not contain "Status", it creates a new key-value pair where the key is the last part of the original key (after splitting it by ".") and the value is the Value field of the original value.

1. `from_entries`: This converts the array of key-value pairs back into a JSON object.

In [4]:
export output=$(echo $json | jq '(.payload 
        | to_entries 
        | map(
            if .key | test("Status") then
              {"key": "NodeId", "value": .key},
              {"key": (.key | split(".")[-1]), "value": .value.Value}
            else
              .
            end
          ) 
        | from_entries)')

echo $output | jq

{
  "NodeId": "ns=2;s=AreaName.MachineName.Status.Idle",
  "Idle": true
}
  "NodeId": "ns=2;s=AreaName.MachineName.Status.Idle",
  "Idle": true
}


Now that we have flattened out the input payload, we can run a JQ query to calculate the machine status. The machine status is calculated based on the "MachineStatus" key (eg. `Idle`). The below JQ query creates a new JSON object with two properties: `NodeId` and `MACHINE_STATUS`. The value of `NodeId` is copied from the input JSON. The value of `MACHINE_STATUS` is then determind by a series of conditional checks based on the value of `MachineStatus` in the input JSON.

In [5]:
echo $output | jq '{
              "NodeId": .NodeId,
              "MACHINE_STATUS": (
                if .Fault == true then "FAULT"
                elif .Idle == true then "IDLE"
                elif .Mode1 == true and .Mode2 == false then "MODE1"
                elif .Mode1 == false and .Mode2 == true then "MODE2"
                else "UNDEFINED"
                end) 
            }'

{
  "NodeId": "ns=2;s=AreaName.MachineName.Status.Idle",
  "MACHINE_STATUS": "IDLE"
}
  "NodeId": "ns=2;s=AreaName.MachineName.Status.Idle",
  "MACHINE_STATUS": "IDLE"
}
